# データセット処理とモデル構築

Hugging Face Hubからデータセットを読み込み、前処理を行い、モデルを構築します。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [ ]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

## 2. ライブラリのインストール

In [ ]:
!pip install -q transformers datasets

## 3. ライブラリのインポート

In [ ]:
from typing import Optional, Dict, Any
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    PreTrainedTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoModelForQuestionAnswering,
    AutoModelForCausalLM,
    AutoConfig,
    PreTrainedModel
)
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 4. DatasetLoaderクラスの定義

In [ ]:
class DatasetLoader:
    """
    Hugging Face データセットのローダークラス

    Hugging Face Hubからデータセットをダウンロードし、
    トークナイズなどの前処理を行います。

    Attributes:
        dataset_name (str): データセット名
        tokenizer (PreTrainedTokenizer): トークナイザー
        max_length (int): 最大シーケンス長
        subset (str): データセットのサブセット名
    """

    def __init__(
        self,
        dataset_name: str,
        tokenizer: PreTrainedTokenizer,
        max_length: int = 512,
        subset: Optional[str] = None
    ):
        """
        Args:
            dataset_name: Hugging Face Hubのデータセット名
            tokenizer: トークナイザー
            max_length: 最大シーケンス長
            subset: データセットのサブセット名
        """
        self.dataset_name = dataset_name
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.subset = subset

    def load(self) -> DatasetDict:
        """
        データセットを読み込む

        Hugging Face Hubからデータセットをダウンロードします。
        初回はインターネット接続が必要で、キャッシュに保存されます。

        Returns:
            DatasetDict: 読み込んだデータセット（train, test等を含む）

        Raises:
            ValueError: データセットが見つからない場合
        """
        logger.info(f"Loading dataset: {self.dataset_name}")

        if self.subset:
            dataset = load_dataset(self.dataset_name, self.subset)
        else:
            dataset = load_dataset(self.dataset_name)

        logger.info(f"Dataset loaded: {dataset}")
        return dataset

    def preprocess_function(self, examples: Dict[str, Any]) -> Dict[str, Any]:
        """
        データの前処理（テキスト分類タスク用の例）

        Args:
            examples: バッチデータ

        Returns:
            トークナイズされたデータ
        """
        # テキストカラムを自動検出
        text_columns = ["text", "sentence", "content", "review", "description", "question"]
        text_column = None
        for col in text_columns:
            if col in examples:
                text_column = col
                break

        if text_column is None:
            available_cols = list(examples.keys())
            raise ValueError(
                f"テキストカラムが見つかりません。"
                f"期待されるカラム: {text_columns}, "
                f"利用可能なカラム: {available_cols}"
            )

        return self.tokenizer(
            examples[text_column],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
        )

    def prepare_dataset(
        self,
        dataset: DatasetDict,
        num_proc: int = 4,
        remove_columns: Optional[list] = None
    ) -> DatasetDict:
        """
        データセットの前処理を適用

        Args:
            dataset: 処理対象のデータセット
            num_proc: 並列処理数
            remove_columns: 削除するカラム名のリスト

        Returns:
            前処理済みデータセット
        """
        logger.info("Preprocessing dataset...")

        # デフォルトで削除するカラムを設定
        if remove_columns is None and "train" in dataset:
            remove_columns = [
                col for col in dataset["train"].column_names
                if col not in ["label", "labels"]
            ]

        processed_dataset = dataset.map(
            self.preprocess_function,
            batched=True,
            num_proc=num_proc,
            remove_columns=remove_columns,
        )

        logger.info("Dataset preprocessing completed")
        return processed_dataset

## 5. CustomDatasetBuilderクラスの定義

In [ ]:
class CustomDatasetBuilder:
    """カスタムデータセットビルダー"""

    @staticmethod
    def from_csv(
        train_path: str,
        validation_path: Optional[str] = None,
        test_path: Optional[str] = None
    ) -> DatasetDict:
        """
        CSVファイルからデータセットを作成

        Args:
            train_path: 訓練データのパス
            validation_path: 検証データのパス
            test_path: テストデータのパス

        Returns:
            DatasetDict
        """
        datasets = {}

        datasets["train"] = Dataset.from_csv(train_path)

        if validation_path:
            datasets["validation"] = Dataset.from_csv(validation_path)

        if test_path:
            datasets["test"] = Dataset.from_csv(test_path)

        return DatasetDict(datasets)

    @staticmethod
    def from_json(
        train_path: str,
        validation_path: Optional[str] = None,
        test_path: Optional[str] = None
    ) -> DatasetDict:
        """
        JSONファイルからデータセットを作成

        Args:
            train_path: 訓練データのパス
            validation_path: 検証データのパス
            test_path: テストデータのパス

        Returns:
            DatasetDict
        """
        datasets = {}

        datasets["train"] = Dataset.from_json(train_path)

        if validation_path:
            datasets["validation"] = Dataset.from_json(validation_path)

        if test_path:
            datasets["test"] = Dataset.from_json(test_path)

        return DatasetDict(datasets)

## 6. ModelBuilderクラスの定義

In [ ]:
class ModelBuilder:
    """Hugging Faceモデルのビルダークラス"""

    @staticmethod
    def build_sequence_classification_model(
        model_name: str,
        num_labels: int,
        pretrained: bool = True,
        config_overrides: Optional[dict] = None
    ) -> PreTrainedModel:
        """
        テキスト分類モデルを構築

        Args:
            model_name: モデル名（例: "bert-base-uncased"）
            num_labels: ラベル数
            pretrained: 事前学習済みモデルを使用するか
            config_overrides: 設定のオーバーライド

        Returns:
            分類モデル
        """
        logger.info(f"Building sequence classification model: {model_name}")

        try:
            config = AutoConfig.from_pretrained(model_name)
        except OSError as e:
            logger.error(f"Model '{model_name}' not found")
            logger.info("Available models: https://huggingface.co/models")
            raise ValueError(f"Invalid model name: {model_name}") from e

        config.num_labels = num_labels

        if config_overrides:
            for key, value in config_overrides.items():
                setattr(config, key, value)

        try:
            if pretrained:
                model = AutoModelForSequenceClassification.from_pretrained(
                    model_name,
                    config=config,
                    ignore_mismatched_sizes=True
                )
            else:
                model = AutoModelForSequenceClassification.from_config(config)
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise

        logger.info(f"Model built successfully with {num_labels} labels")
        return model

    @staticmethod
    def build_token_classification_model(
        model_name: str,
        num_labels: int,
        pretrained: bool = True,
        config_overrides: Optional[dict] = None
    ) -> PreTrainedModel:
        """
        トークン分類モデルを構築（NERなど）

        Args:
            model_name: モデル名
            num_labels: ラベル数
            pretrained: 事前学習済みモデルを使用するか
            config_overrides: 設定のオーバーライド

        Returns:
            トークン分類モデル
        """
        logger.info(f"Building token classification model: {model_name}")

        try:
            config = AutoConfig.from_pretrained(model_name)
        except OSError as e:
            logger.error(f"Model '{model_name}' not found")
            logger.info("Available models: https://huggingface.co/models")
            raise ValueError(f"Invalid model name: {model_name}") from e

        config.num_labels = num_labels

        if config_overrides:
            for key, value in config_overrides.items():
                setattr(config, key, value)

        try:
            if pretrained:
                model = AutoModelForTokenClassification.from_pretrained(
                    model_name,
                    config=config,
                    ignore_mismatched_sizes=True
                )
            else:
                model = AutoModelForTokenClassification.from_config(config)
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise

        logger.info("Token classification model built successfully")
        return model

    @staticmethod
    def build_qa_model(
        model_name: str,
        pretrained: bool = True,
        config_overrides: Optional[dict] = None
    ) -> PreTrainedModel:
        """
        質問応答モデルを構築

        Args:
            model_name: モデル名
            pretrained: 事前学習済みモデルを使用するか
            config_overrides: 設定のオーバーライド

        Returns:
            質問応答モデル
        """
        logger.info(f"Building QA model: {model_name}")

        try:
            config = AutoConfig.from_pretrained(model_name)
        except OSError as e:
            logger.error(f"Model '{model_name}' not found")
            logger.info("Available models: https://huggingface.co/models")
            raise ValueError(f"Invalid model name: {model_name}") from e

        if config_overrides:
            for key, value in config_overrides.items():
                setattr(config, key, value)

        try:
            if pretrained:
                model = AutoModelForQuestionAnswering.from_pretrained(
                    model_name,
                    config=config,
                    ignore_mismatched_sizes=True
                )
            else:
                model = AutoModelForQuestionAnswering.from_config(config)
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise

        logger.info("QA model built successfully")
        return model

    @staticmethod
    def build_causal_lm_model(
        model_name: str,
        pretrained: bool = True,
        config_overrides: Optional[dict] = None
    ) -> PreTrainedModel:
        """
        因果言語モデルを構築（GPTなど）

        Args:
            model_name: モデル名
            pretrained: 事前学習済みモデルを使用するか
            config_overrides: 設定のオーバーライド

        Returns:
            因果言語モデル
        """
        logger.info(f"Building Causal LM model: {model_name}")

        try:
            config = AutoConfig.from_pretrained(model_name)
        except OSError as e:
            logger.error(f"Model '{model_name}' not found")
            logger.info("Available models: https://huggingface.co/models")
            raise ValueError(f"Invalid model name: {model_name}") from e

        if config_overrides:
            for key, value in config_overrides.items():
                setattr(config, key, value)

        try:
            if pretrained:
                model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    config=config
                )
            else:
                model = AutoModelForCausalLM.from_config(config)
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise

        logger.info("Causal LM model built successfully")
        return model

## 7. 使用例

In [ ]:
# 使用例
from transformers import AutoTokenizer

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# DatasetLoaderの使用
loader = DatasetLoader("imdb", tokenizer)
dataset = loader.load()
print(f"読み込んだデータセット: {dataset}")

# 前処理
processed_dataset = loader.prepare_dataset(dataset)
print(f"前処理済みデータセット: {processed_dataset}")

# ModelBuilderの使用
model = ModelBuilder.build_sequence_classification_model(
    model_name="distilbert-base-uncased",
    num_labels=2,
    pretrained=True
)
print(f"モデル: distilbert-base-uncased")
print(f"パラメータ数: {model.num_parameters():,}")
print(f"ラベル数: 2")